In [1]:
%matplotlib widget

In [2]:
from matplotlib import pyplot as plot

In [3]:
import numpy

In [4]:
import torch
from torch import nn
from torch import distributions
from torch import optim

In [326]:
import itertools

In [220]:
class GKF(nn.Module):
    
    def __init__(self, x_dim, h_dim): 
        super(GKF, self).__init__()
        
        # we assume an identity covariance in p(x_t|h_t) and p(h_t|h_{t-1})
        self.transition = nn.Linear(h_dim, h_dim)
        self.emission = nn.Linear(h_dim, x_dim)
        self.initial = nn.Parameter(torch.zeros(h_dim))
        
        self.h_dim = h_dim
        self.x_dim = x_dim
        
    def joint_likelihood(self, x, z):
        assert type(x) == list, 'x must be a list of vectors'
        assert type(z) == list, 'z must be a list of vectors'
        assert len(x)+1 == len(z), 'z must have one more element than x does'
        
        T = len(x)
        
        logp = self._compute_normal_log_p(z[0], self.initial, torch.ones(self.h_dim))
        
        for t in range(T):
            mu_z = self.transition(z[t])
            logp += self._compute_normal_log_p(z[t+1], mu_z, torch.ones(self.h_dim))
            mu_x = self.emission(z[t+1])
            logp += self._compute_normal_log_p(x[t], mu_x, torch.ones(self.x_dim))
            
        return logp
    
    def emit(self, z_list, sample=False):
        x_list = []
        
        for z in z_list:
            mu_x = self.emission(z)
            if sample:
                x = distributions.normal.Normal(mu_x, torch.ones(self.x_dim)).sample()
            else:
                x = mu_x
            x_list.append(x)
            
        return x_list
            
    def sample(self, T, z0=None):
        if z0 is None:
            z0 = distributions.normal.Normal(self.initial, torch.ones(self.h_dim)).sample()
            
        z_list = [z0]
        z = z0
        
        for t in range(T):
            mu_z = self.transition(z)
            z = distributions.normal.Normal(mu_z, torch.ones(self.h_dim)).sample()
            z_list.append(z)

        x_list = self.emit(z_list[1:], sample=True)
            
        return x_list, z_list
        
    def _compute_normal_log_p(self, x, mu, diag_cov):
        return (-(((x - mu) ** 2) / diag_cov) - torch.log(numpy.sqrt(2.) * torch.sqrt(diag_cov))).sum()

In [287]:
class GKFQ(nn.Module):
    def __init__(self, h_dim, T):
        super(GKFQ, self).__init__()
        
        self.diag_cov = nn.ParameterList()
        self.mean = nn.ParameterList()
        
        for t in range(T):
            self.diag_cov.append(nn.Parameter(torch.zeros(h_dim)))
            self.mean.append(nn.Parameter(torch.zeros(h_dim)))
            
        self.T = T
        self.h_dim = h_dim
        
    def sample(self):
        samples = []
        
        for t in range(self.T):
            ss = distributions.normal.Normal(self.mean[t], torch.sqrt(torch.exp(self.diag_cov[t]))).rsample()
            samples.append(ss)
            
        return samples
    
    def compute_log_p(self, z_list):
        log_ps = []
        for t in range(self.T):
            log_ps.append(distributions.normal.Normal(self.mean[t], torch.sqrt(torch.exp(self.diag_cov[t]))).log_prob(z_list[t]))
        return log_ps
    
    def compute_entropy(self):
        entropy = 0.
        for t in range(self.T):
            entropy += distributions.normal.Normal(self.mean[t], torch.sqrt(torch.exp(self.diag_cov[t]))).entropy().sum()
        return entropy

In [350]:
x_dim = 2
h_dim = 2
T = 15

In [351]:
gkf_target = GKF(x_dim, h_dim)

In [352]:
x_list, z_list = gkf_model.sample(T)

In [353]:
''' inference '''
gkf_model = GKF(x_dim, h_dim)
q_model = GKFQ(h_dim, len(x_list)+1)

In [354]:
optimizer = optim.SGD(itertools.chain(q_model.parameters(), gkf_model.parameters()), lr=0.01)

In [355]:
running_loss = None

n_iter = 10000
disp_int = 100
n_samples = 1
entropy_beta = 1.

for ni in range(n_iter):
    optimizer.zero_grad()

    loss = 0.
    for si in range(n_samples):
        z_inferred = q_model.sample()
        loss = loss - gkf_model.joint_likelihood(x_list, z_inferred)
    if running_loss is None:
        running_loss = loss
    else:
        running_loss = 0.9 * running_loss + 0.1 * loss
    if numpy.mod(ni+1, disp_int) == 0:
        print('loss ', running_loss.item())
    loss = loss - entropy_beta * q_model.compute_entropy()
    loss.backward()
    optimizer.step()

loss  96.62820434570312
loss  76.68312072753906
loss  65.92631530761719
loss  58.768375396728516
loss  58.24504852294922
loss  58.106773376464844
loss  59.2044563293457
loss  56.15838623046875
loss  57.45885467529297
loss  55.238975524902344
loss  57.194847106933594
loss  55.231842041015625
loss  55.54340362548828
loss  54.335243225097656
loss  55.999942779541016
loss  57.852294921875
loss  53.83637619018555
loss  56.09925842285156
loss  54.013980865478516
loss  55.39480972290039
loss  54.55963134765625
loss  57.07557678222656
loss  55.67400360107422
loss  57.45320129394531
loss  57.51124572753906
loss  54.596675872802734
loss  53.58145523071289
loss  57.776275634765625
loss  55.404422760009766
loss  57.40546798706055
loss  56.20884704589844
loss  55.90972137451172
loss  56.97712707519531
loss  59.547306060791016
loss  57.963172912597656
loss  56.10658264160156
loss  57.322914123535156
loss  57.17334747314453
loss  58.74747085571289
loss  55.42803955078125
loss  57.50883865356445
loss 

KeyboardInterrupt: 

In [356]:
list(q_model.parameters())[0].grad

tensor([0., 0.])

In [357]:
q_list = q_model.mean
r_list = gkf_model.emit(q_list[1:])

In [358]:
x_list_ = numpy.array([x.numpy() for x in x_list])
z_list_ = numpy.array([z.numpy() for z in z_list])

r_list_ = numpy.array([r.detach().numpy() for r in r_list])
q_list_ = numpy.array([m.detach().numpy() for m in q_list])

plot.figure()

plot.subplot(1,2,1)
plot.plot(x_list_[:,0], x_list_[:,1], 'b-')
for i, x in enumerate(x_list_):
    plot.text(x[0], x[1], '{}'.format(i), color='b')
    
plot.plot(r_list_[:,0], r_list_[:,1], 'r--')
for i, r in enumerate(r_list_):
    plot.text(r[0], r[1], '{}'.format(i), color='r')

plot.subplot(1,2,2)
plot.plot(z_list_[:,0], z_list_[:,1], 'b-')
for i, z in enumerate(z_list_):
    plot.text(z[0], z[1], '{}'.format(i), color='b')
plot.plot(q_list_[:,0], q_list_[:,1], 'r--')
for i, q in enumerate(q_list_):
    plot.text(q[0], q[1], '{}'.format(i), color='r')

plot.show()

FigureCanvasNbAgg()

In [359]:
plot.figure()

plot.subplot(2,1,1)
plot.imshow(x_list_.T)
plot.colorbar()

plot.subplot(2,1,2)
plot.imshow(r_list_.T)
plot.colorbar()

plot.show()

FigureCanvasNbAgg()

In [323]:
plot.close('all')

In [195]:
x_list_.shape

(11, 2)